## Parameter Scans
This notebook demonstrates how to edit / change parameter scan tasks. While most scans can be done using basico, by using simple loops, here we create the scans for the COPASI `Parameter Scan` task. That means, that these scans can be carried out later, using the COPASI graphical user interface, or the command line interface. 

In [1]:
from basico import *

lets start by using the brusselator example, where a scan task is already set up. Using `get_scan_settings`, we can see all the individual settings: 

In [2]:
load_example('brusselator')
get_scan_settings()

{'update_model': False,
 'scheduled': False,
 'subtask': 'Steady-State',
 'output_during_subtask': False,
 'continue_from_current_state': False,
 'continue_on_error': False,
 'scan_items': [{'type': 'scan',
   'num_steps': 10,
   'log': False,
   'min': 0.5,
   'max': 2.0,
   'values': '',
   'use_values': False,
   'item': '(R1).k1',
   'cn': 'CN=Root,Model=The Brusselator,Vector=Reactions[R1],ParameterGroup=Parameters,Parameter=k1,Reference=Value'}]}

the dictionary that is returned, contains all the information stored in the COPASI file. Altenatively you could also just retrieve the scan items using `get_scan_items`. 

In [3]:
get_scan_items()

[{'type': 'scan',
  'num_steps': 10,
  'log': False,
  'min': 0.5,
  'max': 2.0,
  'values': '',
  'use_values': False,
  'item': '(R1).k1',
  'cn': 'CN=Root,Model=The Brusselator,Vector=Reactions[R1],ParameterGroup=Parameters,Parameter=k1,Reference=Value'}]

### Modifying Scan Settings
Analogue to retrieving the settings, they can be set as well, using the same keys, as displayed above. Again, you can change all settings using `set_scan_settings`. Or just change the scan_items using `set_scan_items`. If the scan settings dictionary contain a `scan_items` element, or if `set_scan_items` is called, then all scan items are replaced with the ones given. Alternatively `add_scan_item` can be used to add one or more scan items directly. 

Lets change the scan item from above, so that the `(R1).k1` parameter is changed to the specific values of 0.5, 1.0 and 2.0: 

In [4]:
set_scan_items([{'item': '(R1).k1', 'values': [0.5, 1.0, 2]}])

In [5]:
get_scan_items()

[{'type': 'scan',
  'num_steps': 0,
  'log': False,
  'min': 0.0,
  'max': 1.0,
  'values': [0.5, 1.0, 2.0],
  'use_values': True,
  'item': '(R1).k1',
  'cn': 'CN=Root,Model=The Brusselator,Vector=Reactions[R1],ParameterGroup=Parameters,Parameter=k1'}]

scan items can be specified, either through their display name by specifying the `item` key, or by specifying the `cn` directly. 

The scan item can be of one of three types: 

* `scan`: this is the default (so will be used if not specifie), here a model element is varied either through values, or between a specified  `min` and `max` value
* `repeat`: here the subtask is repeated `num_steps` times
* `random`: here the value for the specified model element is sampled from the specified `distribution` (which can be `uniform`, `normal`, `poisson` or `gamma`)

For example to specify a repeat you'd use: 

In [6]:
add_scan_item(type='repeat', num_steps=10)

In [7]:
get_scan_items()

[{'type': 'scan',
  'num_steps': 0,
  'log': False,
  'min': 0.0,
  'max': 1.0,
  'values': [0.5, 1.0, 2.0],
  'use_values': True,
  'item': '(R1).k1',
  'cn': 'CN=Root,Model=The Brusselator,Vector=Reactions[R1],ParameterGroup=Parameters,Parameter=k1'},
 {'type': 'repeat', 'num_steps': 10, 'log': None, 'min': None, 'max': None}]

as you can see, by using `add_scan_items`, the repeat item is added at the end of the scan list. In COPASI, when multiple scan items are defined, the semantics of those is that for each value of scan item 1, all values of scan item 2 are processed. 


For scan items of type `distribution`, the min/max element take up different meaning: 

* `uniform`: here the value is between the min and max value
* `normal`: min=`mean` and max=`standard deviation`
* `poisson`: min=`mean` and max has no meaning
* `gamma`: min=`shape` and max=`scale`

As a last example, lets change it to sample the initial concentration of species `A` from a normal distribution between around 2, and we want to do that 5 times: 

In [8]:
set_scan_items([
    {
        'type': 'repeat',
        'num_steps': 5
    }, 
    {
        'item': '[A]_0', 
        'type':'random', 
        'distribution': 'normal',
        'min': 2, 
        'max': 0.1
    }])

In [9]:
get_scan_items()

[{'type': 'repeat', 'num_steps': 5, 'log': None, 'min': None, 'max': None},
 {'type': 'random',
  'num_steps': 0,
  'log': False,
  'min': 2.0,
  'max': 0.1,
  'distribution': 'normal',
  'item': '[A]_0',
  'cn': 'CN=Root,Model=The Brusselator,Vector=Compartments[compartment],Vector=Metabolites[A],Reference=InitialConcentration'}]

### Runnin a scan:
You can run these scans in basico using the `run_scan` method. Where you can optionally pass along a `settings` parameter to reconfigure the scan, or an `output` selection, to grab some data directly. 

So lets run the configured scan task, changing it to run a the steady state task, collecting the final concentrations of `X` and `Y`: 

In [10]:
run_scan(settings={'subtask': T.STEADY_STATE}, output=['[A]_0', '[X]', '[Y]'])

,[A]_0,[X],[Y]
0,2.041315,2.041315,1.469639
1,1.925778,1.925778,1.557810
2,2.116724,2.116724,1.417283
3,2.088708,2.088708,1.436292
4,1.950328,1.950328,1.538201
